**mnist-多层感知机**

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)#读取数据
#mnist.train.images是一个55000 * 784维的矩阵, mnist.train.labels是一个55000 * 10维的矩阵

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
n_hidden_1 = 512
n_hidden_2 = 256
n_input    = 784
n_classes  = 10

In [4]:
# INPUTS AND OUTPUTS
x = tf.placeholder(tf.float32, [None, n_input]) # 用placeholder先占地方，样本个数不确定为None
y = tf.placeholder(tf.float32, [None, n_classes]) # 用placeholder先占地方，样本个数不确定为None


In [5]:
# w1=tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=0.1))
# w2=tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=0.1))
regularizer = tf.contrib.layers.l2_regularizer(scale=1.0 / 5000)
w1 = tf.get_variable(
        name="w1",
        regularizer=regularizer,
        initializer=tf.random_normal([n_input, n_hidden_1], stddev=0.1))
w2 = tf.get_variable(
        name="w2",
        regularizer=regularizer,
        initializer=tf.random_normal([n_hidden_1, n_hidden_2], stddev=0.1))  
weights = {
    'w1': w1,#按照高斯分布初始化权重
    'w2': w2,
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=0.1))
}
biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    'b2': tf.Variable(tf.zeros([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes],stddev=0.1))
}
print("NETWORK READY")


NETWORK READY


In [6]:
print(w1)


<tf.Variable 'w1:0' shape=(784, 512) dtype=float32_ref>


**加入L2正则**

In [7]:
tf.add_to_collection(tf.GraphKeys.WEIGHTS,w1)
tf.add_to_collection(tf.GraphKeys.WEIGHTS,w2)
# regularizer = tf.contrib.layers.l2_regularizer(scale=1.0 / 50000)
# print(regularizer)
reg_term = tf.contrib.layers.apply_regularization(regularizer)

# loss = (tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=z_3)) +
#         reg_term)

**隐层的激活函数尝试用RELU，输出层用softmax**

In [8]:
def multilayer_perceptron(_X, _weights, _biases): # 前向传播，隐层用relu激活函数
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['w1']), _biases['b1'])) # 隐层
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['w2']), _biases['b2']))
    return (tf.matmul(layer_2, _weights['out']) + _biases['out']) # 返回输出层的结果，得到十个类别的得分值

pred = multilayer_perceptron(x, weights, biases) # 前向传播的预测值
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)+ reg_term) # 交叉熵损失函数，参数分别为预测值pred和实际label值y，reduce_mean为求平均loss
optm = tf.train.GradientDescentOptimizer(0.3).minimize(cost) # 梯度下降优化器
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) # tf.equal()对比预测值的索引和实际label的索引是否一样，一样返回True，不一样返回False
accr = tf.reduce_mean(tf.cast(corr, tf.float32)) # 将pred即True或False转换为1或0,并对所有的判断结果求均值

init = tf.global_variables_initializer()
print("FUNCTIONS READY")


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

FUNCTIONS READY


In [9]:
# 上面神经网络结构定义好之后，下面定义一些超参数
training_epochs = 100 # 所有样本迭代100次
batch_size = 100 # 每进行一次迭代选择100个样本
display_step = 5
# LAUNCH THE GRAPH
sess = tf.Session() # 定义一个Session
sess.run(init) # 在sess里run一下初始化操作

In [10]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size) # 逐个batch的去取数据
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch
    # Display logs per epoch step
    if epoch % display_step == 0:
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
        test_acc = sess.run(accr, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Epoch: %03d/%03d cost: %.9f TRAIN ACCURACY: %.3f TEST ACCURACY: %.3f"
              % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print("DONE")

Epoch: 000/100 cost: 0.654845242 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.951
Epoch: 005/100 cost: 0.390731738 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.980
Epoch: 010/100 cost: 0.286465457 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.981
Epoch: 015/100 cost: 0.210895940 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.983
Epoch: 020/100 cost: 0.190913507 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.973
Epoch: 025/100 cost: 0.164551673 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.973
Epoch: 030/100 cost: 0.097010477 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.984
Epoch: 035/100 cost: 0.073419419 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.984
Epoch: 040/100 cost: 0.065583655 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.982
Epoch: 045/100 cost: 0.050287199 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.984
Epoch: 050/100 cost: 0.060373035 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.980
Epoch: 055/100 cost: 0.047307676 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.981
Epoch: 060/100 cost: 0.036814879 TRAIN ACCURACY: 1.000 TEST ACCURACY: 0.984
Epoch: 065/1

**一层隐层情况下**
   * 神经元，256，在调整学习率时，设置为0.01，最后训练准确率达到0.97，试着将学习率增大为0.5，准确率最后是0.982，有提高
   * 神经元，512，学习率，0.3，准确率，0.983，增加神经元后性能稍有改善
   
**两层隐层情况下**
   * 两个隐层神经元都设置为256，学习率0.3，最后训练准确率0.981，增加隐层后性能反而略有下降
   * 隐层输入权重w初始化为高斯分布，尝试将输出层的权重也初始化为高斯分布,准确率为0.981，在训练集上很早就达到了1的准确率，测试集效果却一直没有提升，可能存在过拟合，尝试加入正则
   * 加入正则试试看，加入L2正则,准确率0.982，增加第一层的神经元为512，准确率为0.983
   * 尝试调节正则项的参数，scale=5.0 / 50000，改为scale=1.0 / 50000，准确率为0.983，准确率没有提升，但是较快到达了0.983的准确率

**上面尝试的是relu激活函数，下面试试tanh** 
   * 在两个隐层都是256情况下，准确率最后达到了0.980，效果没有relu激活函数好
   
**最后代码用两个隐层，激活函数用relu，神经元数第一层512第二层256，学习率0.3，加大惩罚，L2正则参数scale=1.0 / 5000，得出准确率为0.984**

